---
title: "Spotify Project"
subtitle: "Numeric prediction model"
author: "Oleg Ryzhkov (oryzhkov@chicagobooth.edu)"
date: March 13, 2021
output: neural network model 
---

Load the H2O R package.

- R installation instructions are at http://h2o.ai/download  

In [3]:
library(h2o)

In [4]:
# Start up a 1-node H2O server on your local machine, 
# and allow it to use all CPU cores and up to 8GB of memory.

h2o.init(nthreads=-1, max_mem_size="8G")
h2o.no_progress()

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         7 days 12 hours 
    H2O cluster timezone:       America/Chicago 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.32.0.1 
    H2O cluster version age:    5 months and 4 days !!! 
    H2O cluster name:           H2O_started_from_R_olegryzhkov_tpd759 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   1.71 GB 
    H2O cluster total cores:    8 
    H2O cluster allowed cores:  1 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    H2O API Extensions:         Amazon S3, XGBoost, Algos, AutoML, Core V3, TargetEncoder, Core V4 
    R Version:                  R version 4.0.2 (2020-06-22) 


Warning message in h2o.clusterInfo():
“
Your H2O cluster version is too old (5 months and 4 days)!
Please download and install the latest version from http://h2o.ai/download/”


The `h2o.deeplearning` function fits H2O's 
Deep Learning models from within R.
```{r}
help(h2o.deeplearning)
```

While H2O Deep Learning has many parameters, it is quite easy to use.
Most often we can use the default parameters for many of the inputs.
We should change the following parameters:

- the number and sizes of hidden layers;
- the number of epochs and the activation function;
- add some regularization technique.
  
  
We will apply it to `target_numeric_popularity` data, which
we got cleaning and transforming the original dataset.

In [15]:
# download the file if it does not exist
# if (!file.exists("target_numeric_popularity.csv"))
#   download.file('https://github.com/OlegRyzhkov2020/SpotifyPopularity/blob/main/data/target_numeric_popularity.csv', 'target_numeric_popularity.csv')

target.df <- h2o.importFile('https://github.com/OlegRyzhkov2020/SpotifyPopularity/blob/main/data/target_numeric_popularity.csv') 
dim(target.df)
head(target.df)

[1] 795   3

,C1,C2,C3
,<fct>,<fct>,<fct>
1,<!DOCTYPE,html>,NA
2,<html,"lang=""en""",>
3,<head>,NA,NA
4,<meta,"charset=""utf-8"">",NA
5,<link,"rel=""dns-prefetch""","href=""https://github.githubassets.com"">"
6,<link,"rel=""dns-prefetch""","href=""https://avatars.githubusercontent.com"">"


In [16]:
str(target.df)

Class 'H2OFrame' <environment: 0x7fe304039190> 
 - attr(*, "op")= chr "Parse"
 - attr(*, "id")= chr "target_numeric_popularity.hex_sid_9f3c_3"
 - attr(*, "eval")= logi FALSE
 - attr(*, "nrow")= int 795
 - attr(*, "ncol")= int 3
 - attr(*, "types")=List of 3
  ..$ : chr "enum"
  ..$ : chr "enum"
  ..$ : chr "enum"
 - attr(*, "data")='data.frame':	10 obs. of  3 variables:
  ..$ C1: Factor w/ 174 levels "\n            data-targets=",..: 8 57 55 68 66 66 66 66 66 66
  ..$ C2: Factor w/ 258 levels "\n    data-commit-hovercards-enabled\n    data-discussion-hovercards-enabled\n    data-issue-and-pr-hovercards-"| __truncated__,..: 175 190 NA 25 226 226 226 226 119 119
  ..$ C3: Factor w/ 262 levels "#79b8ff;width:",..: NA 5 NA NA 158 148 151 164 192 192
